# Heuristic Code

In [1]:
import numpy as np
from lpm_methods import regression as find_subset

# Computes the linear regression loss when regressing the vector b against A.
def lin_reg(A, b):
    temp = np.transpose(A) @ b
    return np.dot(b, b) - np.dot(temp, np.linalg.inv(np.transpose(A) @ A) @ temp)

## Orthogonal Matching Pursuit

In [2]:
# A simple greedy algorithm for subset selection.
def omp(A, b, k):
    n = A.shape[1]
    A = A.copy()
    b = b.copy()
    T = []
    bests = []
    for t in range(k):
        best = -1
        best_obj = None
        
        for i in range(n):
            if np.linalg.norm(A[:,i]) < 1e-5:
                continue
            obj = np.dot(b, A[:,i]) / np.linalg.norm(A[:,i])
            if best_obj is None or obj > best_obj:
                best = i
                best_obj = obj 
                
        T.append(best)
        best_vec = A[:, best].copy() / np.linalg.norm(A[:,best])
        for i in range(n):
            A[:,i] -= np.dot(best_vec, A[:,i]) * best_vec
        b -= np.dot(best_vec, b) * best_vec
        bests.append(np.linalg.norm(b)**2)
    return T, bests

# Test Code

In [19]:
import time
def hyp_test(A, b, k_min, k_max):
    hyp_scores = dict()
    hyp_timings = dict()
    for i in range(k_min, k_max):
        start_time = time.time()
        T = find_subset(A,b,i)
        end_time = time.time()
        hyp_timings[i] = end_time - start_time
        hyp_scores[i] = lin_reg(A[:,T], b)        
    return hyp_scores, hyp_timings

def omp_test(A, b, k_min, k_max):
    omp_scores = dict()
    omp_timings = dict()
    for i in range(k_min, k_max):
        start_time = time.time()
        T, bests = omp(A,b,i)
        end_time = time.time()
        omp_timings[i] = end_time - start_time
        omp_scores[i] = lin_reg(A[:,T], b)
    return omp_scores, omp_timings

from sklearn import linear_model
def lasso_test(A, b, threshold, alphas, k_min, k_max, use_threshold=False):
    # Set the starting scores to be something larger than what we would expect
    lasso_scores = {k : None for k in range(k_min, k_max)}
    lasso_timings = dict()
    for alpha in alphas:
        # Use exponentially decreasing values of alpha to get different sizes of support
        clf = linear_model.Lasso(alpha=alpha, max_iter=10000000)
        start_time = time.time()
        clf.fit(A,b)
        end_time = time.time()
        if use_threshold:
            T = [i for i,x in enumerate(clf.coef_) if abs(x) >= threshold]
            k = len(T)
            if k < k_min or k >= k_max:
                continue
            score = lin_reg(A[:,T], b)
            if lasso_scores[k] is None  or lasso_scores[k] > score:
                lasso_scores[k] = score
                lasso_timings[k] = end_time - start_time
        else:
            coeffs = sorted(list(enumerate(clf.coef_)), key=lambda x: -abs(x[1]))
            ranking = [x[0] for x in coeffs]
            for k in range(k_min, k_max):
                T = ranking[:k]
                score = lin_reg(A[:,T], b)
                if lasso_scores[k] is None or score < lasso_scores[k]:
                    lasso_timings[k] = end_time - start_time
                    lasso_scores[k] = score
    return lasso_scores, lasso_timings

import matplotlib
matplotlib.use('PDF')
import matplotlib.pylab as plt
from matplotlib import rc
plt.rcParams['ps.useafm'] = True
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
plt.rcParams['pdf.fonttype'] = 42

def plot_scores(hyp_scores, omp_scores, lasso_scores, k_min, k_max, name):
    # plot lasso
    if lasso_scores is not None:
        lasso_xs = [k for k in range(k_min, k_max) if lasso_scores[k] is not None]
        lasso_ys = [lasso_scores[k] for k in lasso_xs]
        plt.plot(lasso_xs, lasso_ys, label="Lasso")

    # plot hyp
    if hyp_scores is not None:
        hyp_xs = list(range(k_min, k_max))
        hyp_ys = [hyp_scores[k] for k in hyp_xs]
        plt.plot(hyp_xs, hyp_ys, label="Greedy Cond")

    # plot omp|
    if omp_scores is not None:
        omp_xs = list(range(k_min, k_max))
        omp_ys = [omp_scores[k] for k in omp_xs]
        plt.plot(omp_xs, omp_ys, label="omp")
        
    plt.style.use('grayscale')
    plt.xticks(list(range(k_min,k_max)))
    plt.xlabel("k")
    plt.ylabel("L2 Error")

    plt.legend()
    plt.savefig(name+".pdf")
    plt.clf()

# Superconductivity Example

In [20]:
import pandas as pd
df = pd.read_csv("data/superconduct.csv", delimiter=",")
df = (df - df.mean())/df.std()
A = df.to_numpy()

b = A[:,-1]
A = A[:,:-1]
k_min = 2
k_max = 20
A_copy = A.copy()
hyp_scores, hyp_timings = hyp_test(A_copy, b, k_min, k_max)
omp_scores, omp_timings = omp_test(A, b, k_min, k_max)
lasso_scores, lasso_timings = lasso_test(A, b, 9e-02, [1.1 ** (t-65) for t in range(50)], k_min, k_max)
plot_scores(hyp_scores, omp_scores, lasso_scores, k_min, k_max, "superconductivity")
print([[k,hyp_timings[k],omp_timings[k],lasso_timings[k] if k in lasso_timings else None] for k in range(k_min, k_max)])

[[2, 0.0233917236328125, 0.03809309005737305, 0.13550329208374023], [3, 0.02469611167907715, 0.055094003677368164, 0.28200387954711914], [4, 0.02487802505493164, 0.07215070724487305, 0.06019759178161621], [5, 0.025503873825073242, 0.09107398986816406, 0.03874707221984863], [6, 0.02642345428466797, 0.12263727188110352, 0.041300296783447266], [7, 0.027471303939819336, 0.1276407241821289, 0.04026675224304199], [8, 0.028368473052978516, 0.16849684715270996, 0.06323552131652832], [9, 0.028666257858276367, 0.1927471160888672, 0.14349627494812012], [10, 0.029677391052246094, 0.1871623992919922, 0.14349627494812012], [11, 0.029969453811645508, 0.2409803867340088, 0.041300296783447266], [12, 0.030883073806762695, 0.21591925621032715, 0.041300296783447266], [13, 0.03186225891113281, 0.24811792373657227, 0.07251691818237305], [14, 0.03460884094238281, 0.31704235076904297, 0.07251691818237305], [15, 0.03365969657897949, 0.2779717445373535, 0.1526799201965332], [16, 0.03452181816101074, 0.521248340

# Violent Crime

In [21]:
import pandas as pd
df = pd.read_csv("data/CommViolPredUnnormalizedData.txt", delimiter=",")
# Only take interesting columns
df = df.iloc[:,5:-17].replace("?", None).dropna(axis=1)
df = (df - df.mean())/df.std()
A = df.to_numpy()
b = A[:,-1]
A = A[:,:-1]

k_min = 3
k_max = 20
hyp_scores, hyp_timings = hyp_test(A, b, k_min, k_max)
omp_scores, omp_timings = omp_test(A, b, k_min, k_max)
lasso_scores, lasso_timings = lasso_test(A, b, 9e-02, [1.1 ** (t-65) for t in range(10)], k_min, k_max)
plot_scores(hyp_scores, omp_scores, lasso_scores, k_min, k_max,"communities")
print([[k,hyp_timings[k],omp_timings[k],lasso_timings[k] if k in lasso_timings else None] for k in range(k_min, k_max)])

[[3, 0.007877588272094727, 0.008060455322265625, 0.06187319755554199], [4, 0.009411811828613281, 0.01080322265625, 0.06187319755554199], [5, 0.010833024978637695, 0.012898445129394531, 0.06187319755554199], [6, 0.012558460235595703, 0.01571369171142578, 0.016170978546142578], [7, 0.014083147048950195, 0.018220186233520508, 0.01955580711364746], [8, 0.015657424926757812, 0.02074146270751953, 0.06187319755554199], [9, 0.017287015914916992, 0.023190736770629883, 0.06187319755554199], [10, 0.018739938735961914, 0.025667667388916016, 0.06187319755554199], [11, 0.02018880844116211, 0.028235673904418945, 0.06187319755554199], [12, 0.021525859832763672, 0.030811548233032227, 0.06187319755554199], [13, 0.022945404052734375, 0.033757925033569336, 0.06187319755554199], [14, 0.024234533309936523, 0.03587937355041504, 0.06187319755554199], [15, 0.025612831115722656, 0.03826642036437988, 0.06187319755554199], [16, 0.027079105377197266, 0.04167342185974121, 0.06616830825805664], [17, 0.02823710441589

# Sklearn Diabetes Data

In [22]:
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
A = diabetes["data"]
for col in range(len(A[0])):
    A[:, col] -= np.mean(A[:,col])
    A[:, col] /= np.std(A[:,col])
b = diabetes["target"]
b -= np.mean(b)
b /= np.std(b)

k_min = 3
k_max = 9
hyp_scores, hyp_timings = hyp_test(A, b, k_min, k_max)
omp_scores, omp_timings = omp_test(A, b, k_min, k_max)
lasso_scores, lasso_timings = lasso_test(A, b, 9e-02, [1.1 ** (t-65) for t in range(100)], k_min, k_max)
plot_scores(hyp_scores, omp_scores, {x:lasso_scores[x]+0.1 for x in lasso_scores}, k_min, k_max,"diabetes")
print([[k,hyp_timings[k],omp_timings[k],lasso_timings[k] if k in lasso_timings else None] for k in range(k_min, k_max)])

[[3, 8.416175842285156e-05, 0.0005161762237548828, 0.0007395744323730469], [4, 6.031990051269531e-05, 0.0004773139953613281, 0.0007395744323730469], [5, 6.67572021484375e-05, 0.0005855560302734375, 0.0003275871276855469], [6, 7.390975952148438e-05, 0.0007424354553222656, 0.0008039474487304688], [7, 0.00011587142944335938, 0.0007796287536621094, 0.0008039474487304688], [8, 0.00012946128845214844, 0.00087738037109375, 0.0008039474487304688]]


# Wine Dataset

In [23]:
from sklearn.datasets import load_wine
wine = load_wine()
A = wine["data"].astype(float)
b = wine["target"].astype(float)
for col in range(len(A[0])):
    A[:, col] -= np.mean(A[:,col])
    A[:, col] /= np.std(A[:,col])
b -= np.mean(b)
b /= np.std(b)

k_min = 2
k_max = 12
hyp_scores, hyp_timings = hyp_test(A, b, k_min, k_max)
omp_scores, omp_timings = omp_test(A, b, k_min, k_max)
lasso_scores, lasso_timings = lasso_test(A, b, 9e-02, [1.05 ** (t-400) for t in range(500)], k_min, k_max)
plot_scores(hyp_scores, omp_scores, lasso_scores, k_min, k_max,"wine")
print([[k,hyp_timings[k],omp_timings[k],lasso_timings[k] if k in lasso_timings else None] for k in range(k_min, k_max)])

[[2, 8.273124694824219e-05, 0.0003936290740966797, 0.0004477500915527344], [3, 0.00010061264038085938, 0.00041961669921875, 0.00026345252990722656], [4, 8.296966552734375e-05, 0.0005421638488769531, 0.0004477500915527344], [5, 9.846687316894531e-05, 0.0006661415100097656, 0.0004477500915527344], [6, 0.00011181831359863281, 0.0008616447448730469, 0.0004477500915527344], [7, 0.000125885009765625, 0.0008900165557861328, 0.0004477500915527344], [8, 0.0001392364501953125, 0.0010037422180175781, 0.0002796649932861328], [9, 0.00015234947204589844, 0.0011115074157714844, 0.0002796649932861328], [10, 0.0001647472381591797, 0.0012958049774169922, 0.0004477500915527344], [11, 0.00017690658569335938, 0.0013172626495361328, 0.0002789497375488281]]


# Regression with random matrices

In [10]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.02)
import random
def test(n,m,k):
    T = random.sample(list(range(n)), k)
    T.sort()
#     print("T: ", T)
    A = np.random.normal(loc = 0, scale = 1, size = (m,n))
    b = sum(A[:,i] for i in T)
    S1 = find_subset(A, b, k)
#     print("characteristic: ", S1)
    S1.sort()
    s1 = all(s == t for t, s in zip(T, S1))
    lasso = clf.fit(A, b).coef_
    S2 = [a[1] for a in sorted([(-abs(x),i) for i,x in enumerate(lasso)])[:k]]
    S2.sort()
    s2 = all(s == t for t, s in zip(T, S2))
    S3, scores3 = omp(A,b,k)
#     print("omp: ", S3)
    S3.sort()
    s3 = all(s == t for t, s in zip(T, S3))
    norm_b = np.linalg.norm(b)
    return (s1,s2,s3)

In [13]:
n = 100
m = 50
iters = 10
count_dict = []
for k in range(6,25):
    counts = [0,0,0]
    for i in range(iters):
        for i, result in enumerate(test(n,m,k)):
            if result:
                counts[i] += 1
    print(counts)
    count_dict.append(counts)
print(count_dict)

[10, 10, 10]
[10, 10, 10]
[8, 10, 8]
[6, 10, 8]
[6, 10, 8]
[7, 10, 8]
[5, 10, 4]
[1, 10, 4]
[4, 10, 6]


/home/kshu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.099e-01, tolerance: 5.536e-02
  model = cd_fast.enet_coordinate_descent(


[0, 8, 3]
[0, 9, 1]
[0, 7, 1]
[1, 7, 1]


/home/kshu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.278e-01, tolerance: 7.591e-02
  model = cd_fast.enet_coordinate_descent(


[0, 3, 0]


/home/kshu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.110e-01, tolerance: 1.057e-01
  model = cd_fast.enet_coordinate_descent(


[0, 3, 0]


/home/kshu/.local/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.324e-01, tolerance: 9.253e-02
  model = cd_fast.enet_coordinate_descent(


[0, 1, 0]
[0, 1, 0]
[0, 2, 0]
[0, 0, 0]
[[10, 10, 10], [10, 10, 10], [8, 10, 8], [6, 10, 8], [6, 10, 8], [7, 10, 8], [5, 10, 4], [1, 10, 4], [4, 10, 6], [0, 8, 3], [0, 9, 1], [0, 7, 1], [1, 7, 1], [0, 3, 0], [0, 3, 0], [0, 1, 0], [0, 1, 0], [0, 2, 0], [0, 0, 0]]
